**Import libraries**

In [1]:
import numpy as np
import pandas as pd
from google.colab import drive
import zipfile
import math
import datetime
from datetime import datetime
import multiprocessing as mp
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import json
from scipy.optimize import minimize
TOLERANCE = 1e-10
import pickle
import statistics
import matplotlib.pyplot as plt
from matplotlib import pyplot as plt, dates as mdates
from scipy.stats import skew, kurtosis
import statsmodels.api as sm
from scipy import stats
import seaborn as sns

**Connect your drive**

In [2]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Import the monthly and daily data**

In [3]:
archive = zipfile.ZipFile('/content/drive/MyDrive/_Data_/monthly.zip', 'r') #the data is inside a zip file
xlfile = archive.open('monthly.csv')
monthly = pd.read_csv(xlfile, index_col = "date", parse_dates = True).drop(["SHRCD", "EXCHCD", "PERMCO"], axis=1) 

#Clean and prepare the data
monthly["DLRET"] = monthly["DLRET"].fillna(0) 
monthly = monthly[monthly["RET"] != "C"] 
monthly["RET"] = monthly["RET"].astype("float") 
monthly["DLRET"] = monthly["DLRET"].replace(["A", "S"], 0.0) 
monthly["DLRET"] = monthly["DLRET"].astype("float") 
monthly["RET"] = monthly["RET"].fillna(0) 

In [4]:
archive = zipfile.ZipFile('/content/drive/MyDrive/_Data_/daily.zip', 'r')
xlfile = archive.open('daily.csv')
daily = pd.read_csv(xlfile, index_col = "date", parse_dates = True).drop(["SHRCD", "EXCHCD", "PRC"], axis=1)
daily["DLRET"] = daily["DLRET"].fillna(0)
daily = daily[daily["RET"] != "C"]
daily["RET"] = daily["RET"].astype("float")
daily["RET"] = daily["RET"].fillna(0)

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


**Create a size variable with the market cap of each stock**

In [5]:
#Some Prices are negative because bid/ask midpoint prices are reported as negative numbers in CRSP 
monthly["Size"] = abs(monthly["PRC"]) * monthly["SHROUT"]

**Create a list with all the months for the period. This will help map the data**

In [6]:
#The sample starts in 1997, and goes until 2022
years = [str(i) for i in range(1997, 2023)]
months = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]

dates = []
for year in years:
    for month in months:
        date = year + "-" + month
        dates.append(date)
#Date only goes until 2022-03
dates.pop(-1)
dates.pop(-1)
dates.pop(-1)
dates.pop(-1)
dates.pop(-1)
dates.pop(-1)
dates.pop(-1)
dates.pop(-1)
dates.pop(-1)

'2022-04'

In [ ]:
#Save for future use

with open("/content/drive/MyDrive/_Data_/dates.json", "wb") as output:

    pickle.dump(dates, output)

**A list with the final day of each month for the period. Use Microsoft given that has data for every period in the sample**

In [7]:
msft = monthly[monthly["PERMNO"] == 10107]
days = []
for date in msft.index:
    days.append(date)

**Another for the daily days**

In [8]:
#We need this to calculate the betas in a future step
sp500_daily = pd.read_csv('/content/drive/MyDrive/_Data_/sp500_daily.csv', index_col = "date", parse_dates = True)["sprtrn"]
daily_days = []
for date in sp500_daily.index:
  daily_days.append(date)

**Next create a function to get the returns of a given stock for every month in the past year + a function to calculate the cummulative annual return**

In [9]:
def getPastYearReturns(stock, date):
    index = dates.index(date)
    obs = monthly[monthly["PERMNO"] == stock]["RET"][index - 11 : index + 1]
    
    return obs

def cummulativeAnnualReturn(returns):
    cumRet = 1
    for ret in returns:
        cumRet = cumRet * (1 + ret) 
    cumRet -= 1
    return cumRet 

**Get the stocks for each month, while filtering out the last quartile in terms of size**

In [9]:
monthly_stocksAndReturns = []
for day in days:
    df = monthly[monthly.index == day]
    df["Quartile"] = pd.qcut(df['Size'], 4,
                               labels = False) #divide the data into quartiles
    df = df[df["Quartile"] != 0] #filter the bottom quartile
    y = []
    x = df["PERMNO"]
    z = df["RET"]
    stocks = []
    for stock in x:
        stocks.append(stock)
    y.append(stocks)
    y.append(z)
    monthly_stocksAndReturns.append(y)
    print((days.index(day) + 1) / len(days) * 100) #to update the progress of the code

<ipython-input-9-8f37e7e307a4>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Quartile"] = pd.qcut(df['Size'], 4,


0.33003300330033003
0.6600660066006601
0.9900990099009901
1.3201320132013201
1.65016501650165
1.9801980198019802
2.31023102310231
2.6402640264026402
2.9702970297029703
3.3003300330033
3.6303630363036308
3.9603960396039604
4.29042904290429
4.62046204620462
4.9504950495049505
5.2805280528052805
5.6105610561056105
5.9405940594059405
6.270627062706271
6.6006600660066
6.9306930693069315
7.2607260726072615
7.590759075907591
7.920792079207921
8.25082508250825
8.58085808580858
8.91089108910891
9.24092409240924
9.570957095709572
9.900990099009901
10.231023102310232
10.561056105610561
10.891089108910892
11.221122112211221
11.55115511551155
11.881188118811881
12.211221122112212
12.541254125412541
12.871287128712872
13.2013201320132
13.531353135313532
13.861386138613863
14.19141914191419
14.521452145214523
14.85148514851485
15.181518151815181
15.51155115511551
15.841584158415841
16.17161716171617
16.5016501650165
16.831683168316832
17.16171617161716
17.491749174917494
17.82178217821782
18.15181518

**Another list, but for annual returns (Estimated Time: 30-40 minutes).**

In [ ]:
annualRets = []
for i in range(12, len(dates)):
    rets = []
    stocks = monthly_stocksAndReturns[i][0]
    for stock in stocks:
        try:
            annRet = cummulativeAnnualReturn(getPastYearReturns(stock, dates[i]))
        except:
            annRet = 0
        rets.append(annRet)
    annualRets.append(rets)
    print((i + 1) / len(dates) * 100)

In [12]:
#Save the list to avoid make the process all over again
with open("/content/drive/MyDrive/_Data_/annualReturns.json", "wb") as output:

    pickle.dump(annualRets, output)

In [10]:
#Load if you're re-running the code
with open("/content/drive/MyDrive/_Data_/annualReturns.json", "rb") as data:
    annualRets = pickle.load(data)

**Now the volatilities**

In [11]:
#Start with the first month daily returns
daily_returns = []
info = daily[ : str(days[0])]
x = []
for stock in monthly_stocksAndReturns[0][0]:
  x.append(info[info["PERMNO"] == stock]["RET"])
daily_returns.append(x)

In [12]:
#Then get daily returns for the other periods
for i in range(1, len(days)):
  info = daily[ str(days[i-1]) : str(days[i])]
  x = []
  for stock in monthly_stocksAndReturns[i][0]:
    x.append(info[info["PERMNO"] == stock]["RET"])
  daily_returns.append(x)
  print((i + 1) / len(days) * 100)

0.6600660066006601
0.9900990099009901
1.3201320132013201
1.65016501650165
1.9801980198019802
2.31023102310231
2.6402640264026402
2.9702970297029703
3.3003300330033
3.6303630363036308
3.9603960396039604
4.29042904290429
4.62046204620462
4.9504950495049505
5.2805280528052805
5.6105610561056105
5.9405940594059405
6.270627062706271
6.6006600660066
6.9306930693069315
7.2607260726072615
7.590759075907591
7.920792079207921
8.25082508250825
8.58085808580858
8.91089108910891
9.24092409240924
9.570957095709572
9.900990099009901
10.231023102310232
10.561056105610561
10.891089108910892
11.221122112211221
11.55115511551155
11.881188118811881
12.211221122112212
12.541254125412541
12.871287128712872
13.2013201320132
13.531353135313532
13.861386138613863
14.19141914191419
14.521452145214523
14.85148514851485
15.181518151815181
15.51155115511551
15.841584158415841
16.17161716171617
16.5016501650165
16.831683168316832
17.16171617161716
17.491749174917494
17.82178217821782
18.151815181518153
18.481848184

In [13]:
#And finally calculate annualized volatility for each stock in each period
volatilities = []
count = 0
for month in daily_returns:
  monthly_vols = []
  for stock in month:
    vol = stock.std() * np.sqrt(252)
    monthly_vols.append(vol)
  volatilities.append(monthly_vols)
  count += 1
  print(count / len(days))

0.0033003300330033004
0.006600660066006601
0.009900990099009901
0.013201320132013201
0.0165016501650165
0.019801980198019802
0.0231023102310231
0.026402640264026403
0.0297029702970297
0.033003300330033
0.036303630363036306
0.039603960396039604
0.0429042904290429
0.0462046204620462
0.04950495049504951
0.052805280528052806
0.056105610561056105
0.0594059405940594
0.0627062706270627
0.066006600660066
0.06930693069306931
0.07260726072607261
0.07590759075907591
0.07920792079207921
0.08250825082508251
0.0858085808580858
0.0891089108910891
0.0924092409240924
0.09570957095709572
0.09900990099009901
0.10231023102310231
0.10561056105610561
0.10891089108910891
0.11221122112211221
0.11551155115511551
0.1188118811881188
0.12211221122112212
0.1254125412541254
0.12871287128712872
0.132013201320132
0.1353135313531353
0.13861386138613863
0.1419141914191419
0.14521452145214522
0.1485148514851485
0.15181518151815182
0.1551155115511551
0.15841584158415842
0.1617161716171617
0.16501650165016502
0.1683168316

**Group all the data into a dictionary**

In [14]:
organized_data = []
#pool = mp.Pool(mp.cpu_count())
for i in range(12, len(dates)):
    y = {
        "Date": dates[i],
        "Stocks": {}
        }
    stocks = monthly_stocksAndReturns[i][0]
    for stock in stocks:
        try:
            x = {
                "Id": stock,
                "Monthly Return": monthly_stocksAndReturns[i][1][stocks.index(stock)],
                "Annual Return": annualRets[i - 12][stocks.index(stock)],
                "Daily Returns": daily_returns[i][stocks.index(stock)],
                "Volatility": volatilities[i][stocks.index(stock)]
            }
            y["Stocks"][stock] = x
        except:
            pass
    print((i+1) / len(dates) * 100)
    organized_data.append(y)

4.29042904290429
4.62046204620462
4.9504950495049505
5.2805280528052805
5.6105610561056105
5.9405940594059405
6.270627062706271
6.6006600660066
6.9306930693069315
7.2607260726072615
7.590759075907591
7.920792079207921
8.25082508250825
8.58085808580858
8.91089108910891
9.24092409240924
9.570957095709572
9.900990099009901
10.231023102310232
10.561056105610561
10.891089108910892
11.221122112211221
11.55115511551155
11.881188118811881
12.211221122112212
12.541254125412541
12.871287128712872
13.2013201320132
13.531353135313532
13.861386138613863
14.19141914191419
14.521452145214523
14.85148514851485
15.181518151815181
15.51155115511551
15.841584158415841
16.17161716171617
16.5016501650165
16.831683168316832
17.16171617161716
17.491749174917494
17.82178217821782
18.151815181518153
18.48184818481848
18.81188118811881
19.141914191419144
19.471947194719473
19.801980198019802
20.13201320132013
20.462046204620464
20.792079207920793
21.122112211221122
21.45214521452145
21.782178217821784
22.112211

**Define the functions for the risk parity weights**

In [ ]:
def _allocation_risk(weights, covariances): 
    portfolio_risk = np.sqrt((weights * covariances * weights.T))[0, 0]
    
    return portfolio_risk
    
def _assets_risk_contribution_to_allocation_risk(weights, covariances):
    portfolio_risk = _allocation_risk(weights, covariances)
    assets_risk_contribution = np.multiply(weights.T, covariances * weights.T) / portfolio_risk
    
    return assets_risk_contribution

def _risk_budget_objective_error(weights, args):
    covariances = args[0]
    asset_risk_budget = args[1]
    weights = np.matrix(weights)
    portfolio_risk = _allocation_risk(weights, covariances)
    assets_risk_contribution = _assets_risk_contribution_to_allocation_risk(weights, covariances)
    
    assets_risk_target = np.asmatrix(np.multiply(portfolio_risk, assets_risk_budget))
    
    error = sum(np.square(assets_risk_contribution - assets_risk_target.T))[0, 0]
    
    return error

def _get_risk_parity_weights(covariances, assets_risk_budget, initial_weights):
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1.0},
                   {'type': 'ineq', 'fun': lambda x: x})
    
    optimize_result = minimize(fun=_risk_budget_objective_error,
                               x0=initial_weights,
                               args=[covariances, assets_risk_budget],
                               method='SLSQP',
                               constraints=constraints,
                               tol=TOLERANCE,
                               options={'disp': False})
    
    weights = optimize_result.x
    
    return weights

**Create the portfolios for each period (NOTE: This is a process that takes some hours. I included a process in the bottom that updates the progress. Be aware that after the process is done, if you don't run any command for some time, the google colab interrupts the session, so run already the next step that saves these portfolios in your drive so you don't lose your progress**

In [ ]:
portfolios = []
for date in organized_data:
  stocks = date["Stocks"].keys()
  y = {
        "Date": date["Date"],
        "Portfolios":{
            "Long Portfolio": {},
            "Short Portfolio": {}
        }
    }
    
  long_stocks = [date["Stocks"][stock]["Id"] for stock in stocks if date["Stocks"][stock]["Annual Return"] > 0]
  short_stocks = [date["Stocks"][stock]["Id"] for stock in stocks if date["Stocks"][stock]["Annual Return"] < 0]
  long_returns = [date["Stocks"][stock]["Annual Return"] for stock in stocks if date["Stocks"][stock]["Annual Return"] > 0]
  short_returns = [date["Stocks"][stock]["Annual Return"] for stock in stocks if date["Stocks"][stock]["Annual Return"] < 0]

  #First for the long stocks

  df = pd.DataFrame()
  df["Stocks"] = long_stocks
  df["Returns"] = long_returns
  df['Quintile_rank'] = pd.qcut(df['Returns'], 10,
                               labels = False)
  df = df[df["Quintile_rank"] == 9] #only select the long decile

  stocks = df["Stocks"].reset_index()["Stocks"]
  y["Portfolios"]["Long Portfolio"]["Stocks"] = stocks #These will be the stocks that make the long leg


  daily_returns = []
  for stock in stocks:
    daily_returns.append(date["Stocks"][stock]["Daily Returns"]) 
  
  df = pd.DataFrame()
  for i in range(len(stocks)):
    df[stocks[i]] = daily_returns[i]

  covariances = 252 * df.cov().values #calculate the covariance matrix based on the daily returns of that month
  initial_weights = [1 / len(stocks)] * len(stocks)
  assets_risk_budget = [1 / len(stocks)] * len(stocks)
  y["Portfolios"]["Long Portfolio"]["Weights"] = _get_risk_parity_weights(covariances, assets_risk_budget, initial_weights) #optimize the weights

  #Now for the short stocks

  df = pd.DataFrame()
  df["Stocks"] = short_stocks
  df["Returns"] = short_returns
  df['Quintile_rank'] = pd.qcut(df['Returns'], 10,
                               labels = False)
  df = df[df["Quintile_rank"] == 0]

  stocks = df["Stocks"].reset_index()["Stocks"]
  y["Portfolios"]["Short Portfolio"]["Stocks"] = stocks


  daily_returns = []
  for stock in stocks:
    daily_returns.append(date["Stocks"][stock]["Daily Returns"])
  
  df = pd.DataFrame()
  for i in range(len(stocks)):
    df[stocks[i]] = daily_returns[i]

  covariances = 252 * df.cov().values
  initial_weights = [1 / len(stocks)] * len(stocks)
  assets_risk_budget = [1 / len(stocks)] * len(stocks)
  y["Portfolios"]["Short Portfolio"]["Weights"] = _get_risk_parity_weights(covariances, assets_risk_budget, initial_weights) 
    
  portfolios.append(y)

  print((organized_data.index(date) + 1) / len(organized_data) * 100)

**SAVE HERE**

In [ ]:
#Save the portfolios to avoid running the code again. You will also need these for the other notebooks.
with open("/content/drive/MyDrive/_Data_/weights.json", "wb") as output:
    pickle.dump(portfolios, output)

**Set a volatility target**

In [15]:
target = 0.18

**Import the risk-free rate, the monthly SP500, Fama-French Factors, and the VIX index**

In [16]:
fff = pd.read_csv('/content/drive/MyDrive/_Data_/ffm.csv', index_col = "dateff", parse_dates = True)[1:]
rf = fff["rf"]

sp500 = pd.read_csv('/content/drive/MyDrive/_Data_/sp500.csv', index_col = "DATE", parse_dates = True)[1:]
vix = pd.read_csv('/content/drive/MyDrive/_Data_/^VIX.csv', index_col = "Date", parse_dates = True)["Adj Close"]

**Create the volatility signal (High if VIX > 30**

In [17]:
signals = []
for value in vix.values:
  if value > 30:
    signals.append("High")
  elif 15 < value <= 30:
    signals.append("Normal")
  else:
    signals.append("Low")

**Load the portfolios data if you are re-running the code**

In [18]:
with open("/content/drive/MyDrive/_Data_/weights.json", "rb") as data:
    portfolios = pickle.load(data)

****Calculate Strategy A returns****

In [ ]:
#I will initiate three lists to save the weights of each leg, and the overall weights
short_weights_historic = []
long_weights_historic = []
total_weights_historics = []
port_rets = []
for i in portfolios:
  date = dates.index(i["Date"]) + 1 #we need the monthly returns of the next month, so we add 1 to the index
  #First for the stocks in the long portfolio
  long_stocks = i["Portfolios"]["Long Portfolio"]["Stocks"]
  long_rets = []
  long_volatilities = []
  
  for stock in long_stocks:
    try:
      long_rets.append((1 + monthly[monthly["PERMNO"] == stock][dates[date]]["RET"].values[0]) * (1 + monthly[monthly["PERMNO"] == stock][dates[date]]["DLRET"].values[0]) - 1) #because some stocks are delisted, the return is the product of the delisting return and the normal return
      long_volatilities.append(target / organized_data[portfolios.index(i)]["Stocks"][stock]["Volatility"]) #individual volatility targetting
    except:
      long_rets.append(0.0) #In the case that some stock has some error 
      long_volatilities.append(target / organized_data[portfolios.index(i)]["Stocks"][stock]["Volatility"])
  long_weights = i["Portfolios"]["Long Portfolio"]["Weights"].tolist() #weights from the risk-parity rule
  long_RET = 0
  long_WEIGHT = 0
  
  for n in range(len(long_weights)):
    long_RET += long_weights[n] * (long_rets[n]) * long_volatilities[n]
    long_WEIGHT += long_weights[n] * long_volatilities[n]
  long_weights_historic.append(long_WEIGHT)

#Same for short portfolio
  short_stocks = i["Portfolios"]["Short Portfolio"]["Stocks"]
  short_rets = []
  short_volatilities = []
  for stock in short_stocks:
    try:
      short_rets.append((1 + monthly[monthly["PERMNO"] == stock][dates[date]]["RET"].values[0]) * (1 + monthly[monthly["PERMNO"] == stock][dates[date]]["DLRET"].values[0]) - 1)
      short_volatilities.append(target / organized_data[portfolios.index(i)]["Stocks"][stock]["Volatility"])
    except:
      short_rets.append(0.0)  
      short_volatilities.append(target / organized_data[portfolios.index(i)]["Stocks"][stock]["Volatility"])
  short_weights = i["Portfolios"]["Short Portfolio"]["Weights"].tolist()
  short_RET = 0
  short_WEIGHT = 0 

  for n in range(len(short_weights)):
    short_RET += -short_weights[n] * (short_rets[n]) * short_volatilities[n]
    short_WEIGHT += -short_weights[n] * short_volatilities[n]
  short_weights_historic.append(short_WEIGHT)

  monthly_return = long_RET + short_RET
  if signals[portfolios.index(i)] == "High": #if volatility is high, we invest in the risk-free rate
    total_weights_historics.append(0.0)
    port_rets.append(0.0)
  else: 
    total_weights_historics.append(long_WEIGHT + short_WEIGHT)  
    port_rets.append(monthly_return)

  print((portfolios.index(i) + 1) / len(portfolios) * 100) 


**Save all the variables required for the other notebooks**

In [22]:
with open("/content/drive/MyDrive/_Data_/port_rets.json", "wb") as output:
    pickle.dump(port_rets, output)

with open("/content/drive/MyDrive/_Data_/long_weights_historic.json", "wb") as output:
    pickle.dump(long_weights_historic, output)

with open("/content/drive/MyDrive/_Data_/short_weights_historic.json", "wb") as output:
    pickle.dump(short_weights_historic, output)

with open("/content/drive/MyDrive/_Data_/total_weights_historic.json", "wb") as output:
    pickle.dump(total_weights_historics, output)


**Strategy B**

In [ ]:
#The three lists for weights + the 2 lists for betas
port_rets_beta = []
beta_long_historic = []
beta_short_historic = []
short_weights_beta_historic = []
long_weights_beta_historic = []
total_weights_beta_historic = []

for i in portfolios:
  date = dates.index(i["Date"]) + 1
  #First for the stocks in the long portfolio
  long_stocks = i["Portfolios"]["Long Portfolio"]["Stocks"]
  long_rets = []
  long_betas = []
  long_volatilities = []
  for stock in long_stocks:
    try:
      long_rets.append((1 + monthly[monthly["PERMNO"] == stock][dates[date]]["RET"].values[0]) * (1 + monthly[monthly["PERMNO"] == stock][dates[date]]["DLRET"].values[0]) - 1)
      long_volatilities.append(target / organized_data[portfolios.index(i)]["Stocks"][stock]["Volatility"])
      #We want to scale the beta for each stock
      stock_returns = daily[daily["PERMNO"] == stock]["RET"][days[date - 13] : days[date - 1]]
      length_stock = len(stock_returns)
      index = daily_days.index(days[date - 1]) + 1
      sp_500 = sp500_daily[index - length_stock : index]
      beta = stats.linregress(sp_500, stock_returns)[0]
      long_betas.append(beta)


    except:
      long_rets.append(0.0)  
      long_volatilities.append(target / organized_data[portfolios.index(i)]["Stocks"][stock]["Volatility"])
      stock_returns = daily[daily["PERMNO"] == stock]["RET"][days[date - 13] : days[date - 1]]
      length_stock = len(stock_returns)
      index = daily_days.index(days[date - 1]) + 1
      sp_500 = sp500_daily[index - length_stock : index]
      beta = stats.linregress(sp_500, stock_returns)[0]
      long_betas.append(beta)

    count += 1
  long_weights = i["Portfolios"]["Long Portfolio"]["Weights"].tolist()
  long_RET = 0
  long_WEIGHT = 0
  
  beta_long = 0
  for n in range(len(long_weights)):
    long_RET += long_weights[n] * (long_rets[n]) * long_volatilities[n]
    beta_long += long_weights[n] * long_betas[n] * long_volatilities[n]
    long_WEIGHT += long_weights[n] * long_volatilities[n]
  beta_long_historic.append(beta_long)

#Same for short portfolio
  short_stocks = i["Portfolios"]["Short Portfolio"]["Stocks"]
  short_rets = []
  short_betas = []
  short_volatilities = []
  for stock in short_stocks:
    try:
      short_rets.append((1 + monthly[monthly["PERMNO"] == stock][dates[date]]["RET"].values[0]) * (1 + monthly[monthly["PERMNO"] == stock][dates[date]]["DLRET"].values[0]) - 1)
      short_volatilities.append(target / organized_data[portfolios.index(i)]["Stocks"][stock]["Volatility"])
      stock_returns = daily[daily["PERMNO"] == stock]["RET"][days[date - 13] : days[date - 1]]
      length_stock = len(stock_returns)
      index = daily_days.index(days[date - 1]) + 1
      sp_500 = sp500_daily[index - length_stock : index]
      beta = stats.linregress(sp_500, stock_returns)[0]
      short_betas.append(beta)

    except:
      short_rets.append(0.0)  
      short_volatilities.append(target / organized_data[portfolios.index(i)]["Stocks"][stock]["Volatility"])
      stock_returns = daily[daily["PERMNO"] == stock]["RET"][days[date - 13] : days[date - 1]]
      length_stock = len(stock_returns)
      index = daily_days.index(days[date - 1]) + 1
      sp_500 = sp500_daily[index - length_stock : index]
      beta = stats.linregress(sp_500, stock_returns)[0]
      short_betas.append(beta)

  short_weights = i["Portfolios"]["Short Portfolio"]["Weights"].tolist()
  short_RET = 0
  short_WEIGHT = 0 
   
  beta_short = 0
  for n in range(len(short_weights)):
    short_RET += -short_weights[n] * (short_rets[n]) * short_volatilities[n]
    beta_short += short_weights[n] * short_volatilities[n] * short_betas[n]
    short_WEIGHT += -short_weights[n] * short_volatilities[n]
  beta_short_historic.append(beta_short)

  beta_long = abs(beta_long)
  beta_short = abs(beta_short)

  if beta_long < 0.5: #Limit leverage to 200%
    beta_long = 0.5
    long_WEIGHT = long_WEIGHT * 2
  else:
    long_WEIGHT = long_WEIGHT * 1/beta_long
  long_weights_beta_historic.append(long_WEIGHT)

  if beta_short < 0.5:
    beta_short = 0.5
    short_WEIGHT = short_WEIGHT * 2
  else:
    short_WEIGHT = short_WEIGHT * 1/beta_short
  short_weights_beta_historic.append(short_WEIGHT)
  
  


  monthly_return = (1 / beta_long) * long_RET + (1 / beta_short) * short_RET

  if signals[portfolios.index(i)] == "High":
    total_weights_beta_historic.append(0.0)
    port_rets_beta.append(0.0)
  else:
    total_weights_beta_historic.append(long_WEIGHT + short_WEIGHT)   
    port_rets_beta.append(monthly_return)
  print((portfolios.index(i) + 1) / len(portfolios) * 100)

**Save Variables**

In [ ]:
with open("/content/drive/MyDrive/_Data_/ports_beta.json", "wb") as output:
    pickle.dump(port_rets_beta, output)


with open("/content/drive/MyDrive/_Data_/beta_long.json", "wb") as output:
    pickle.dump(beta_long_historic, output)

with open("/content/drive/MyDrive/_Data_/beta_short.json", "wb") as output:
    pickle.dump(beta_short_historic, output)

with open("/content/drive/MyDrive/_Data_/long_weights_beta_historic.json", "wb") as output:
    pickle.dump(long_weights_beta_historic, output)
  
with open("/content/drive/MyDrive/_Data_/short_weights_beta_historic.json", "wb") as output:
    pickle.dump(short_weights_beta_historic, output)

with open("/content/drive/MyDrive/_Data_/total_weights_beta_historic.json", "wb") as output:
    pickle.dump(total_weights_beta_historic, output)

**Strategy A limits on leverage**

In [19]:
with open("/content/drive/MyDrive/_Data_/dates.json", "rb") as data:
    plot_dates = pickle.load(data)[12:] #We only have returns starting in the 13th month

In [ ]:
#I will initiate three lists to save the weights of each leg, and the overall weights
short_weights_historic = []
long_weights_historic = []
total_weights_historics = []
port_rets = []
for i in portfolios:
  date = dates.index(i["Date"]) + 1 #we need the monthly returns of the next month, so we add 1 to the index
  #First for the stocks in the long portfolio
  long_stocks = i["Portfolios"]["Long Portfolio"]["Stocks"]
  long_rets = []
  long_volatilities = []
  
  for stock in long_stocks:
    try:
      long_rets.append((1 + monthly[monthly["PERMNO"] == stock][dates[date]]["RET"].values[0]) * (1 + monthly[monthly["PERMNO"] == stock][dates[date]]["DLRET"].values[0]) - 1) #because some stocks are delisted, the return is the product of the delisting return and the normal return
      long_volatilities.append(target / organized_data[portfolios.index(i)]["Stocks"][stock]["Volatility"]) #individual volatility targetting
    except:
      long_rets.append(0.0) #In the case that some stock has some error 
      long_volatilities.append(target / organized_data[portfolios.index(i)]["Stocks"][stock]["Volatility"])
  long_weights = i["Portfolios"]["Long Portfolio"]["Weights"].tolist() #weights from the risk-parity rule
  long_RET = 0
  long_WEIGHT = 0
  
  for n in range(len(long_weights)):
    long_RET += long_weights[n] * (long_rets[n]) * long_volatilities[n]
    long_WEIGHT += long_weights[n] * long_volatilities[n]
  long_weights_historic.append(long_WEIGHT)

#Same for short portfolio
  short_stocks = i["Portfolios"]["Short Portfolio"]["Stocks"]
  short_rets = []
  short_volatilities = []
  for stock in short_stocks:
    try:
      short_rets.append((1 + monthly[monthly["PERMNO"] == stock][dates[date]]["RET"].values[0]) * (1 + monthly[monthly["PERMNO"] == stock][dates[date]]["DLRET"].values[0]) - 1)
      short_volatilities.append(target / organized_data[portfolios.index(i)]["Stocks"][stock]["Volatility"])
    except:
      short_rets.append(0.0)  
      short_volatilities.append(target / organized_data[portfolios.index(i)]["Stocks"][stock]["Volatility"])
  short_weights = i["Portfolios"]["Short Portfolio"]["Weights"].tolist()
  short_RET = 0
  short_WEIGHT = 0 

  for n in range(len(short_weights)):
    short_RET += -short_weights[n] * (short_rets[n]) * short_volatilities[n]
    short_WEIGHT += -short_weights[n] * short_volatilities[n]
  short_weights_historic.append(short_WEIGHT)

  total_weight = short_WEIGHT + long_WEIGHT
  monthly_return = long_RET + short_RET


  if abs(total_weight) > 2.0:
    monthly_return = monthly_return * (2 / abs(total_weight)) #if the leverage is higher than 200% or lower than -200%, deleverage the monthly return
  else:
    pass

  if signals[portfolios.index(i)] == "High": #if volatility is high, we invest in the risk-free rate
    total_weights_historics.append(0.0)
    port_rets.append(0.0)
  else: 
    total_weights_historics.append(long_WEIGHT + short_WEIGHT)  
    port_rets.append(monthly_return)

  print((portfolios.index(i) + 1) / len(portfolios) * 100) 


In [22]:
excess_returns = pd.DataFrame(port_rets[:-1], index = plot_dates[1:], columns = ["Strategy Returns"]) 
excess_returns["Risk Free"] = rf.values
excess_returns["Excess Returns"] = excess_returns["Strategy Returns"] - excess_returns["Risk Free"]
rets = excess_returns["Excess Returns"].values

std = statistics.pstdev(rets) * np.sqrt(12) * 100
mean = np.mean(rets) * 12 * 100
skewness = skew(rets)
kurt = kurtosis(rets)
max = np.max(rets) * 100
min = np.min(rets) * 100
std_negative = statistics.pstdev([ret for ret in rets if ret < 0])  * np.sqrt(12) * 100

print("Standard Deviation: " + str(std) + "%")
print("Average Annual Return: " + str(mean) + "%")
print("Sharpe Ratio: " + str(mean/std))
print("Sortino Ratio: " + str(mean/std_negative))
print("Largest Gain: " + str(max) + "%")
print("Largest Loss: " + str(min) + "%") 
print("Skewness: " + str(skewness))
print("Kurtosis: " + str(kurt))

Standard Deviation: 7.468481901460286%
Average Annual Return: 7.531742462514416%
Sharpe Ratio: 1.0084703373307715
Sortino Ratio: 1.5231172828009496
Largest Gain: 9.314447486529419%
Largest Loss: -9.330472481742518%
Skewness: 0.1451827848194708
Kurtosis: 3.17890529094916


**Strategy B limits on leverage**

In [ ]:
#The three lists for weights + the 2 lists for betas
port_rets_beta = []
beta_long_historic = []
beta_short_historic = []
short_weights_beta_historic = []
long_weights_beta_historic = []
total_weights_beta_historic = []

for i in portfolios:
  date = dates.index(i["Date"]) + 1
  #First for the stocks in the long portfolio
  long_stocks = i["Portfolios"]["Long Portfolio"]["Stocks"]
  long_rets = []
  long_betas = []
  long_volatilities = []
  for stock in long_stocks:
    try:
      long_rets.append((1 + monthly[monthly["PERMNO"] == stock][dates[date]]["RET"].values[0]) * (1 + monthly[monthly["PERMNO"] == stock][dates[date]]["DLRET"].values[0]) - 1)
      long_volatilities.append(target / organized_data[portfolios.index(i)]["Stocks"][stock]["Volatility"])
      #We want to scale the beta for each stock
      stock_returns = daily[daily["PERMNO"] == stock]["RET"][days[date - 13] : days[date - 1]]
      length_stock = len(stock_returns)
      index = daily_days.index(days[date - 1]) + 1
      sp_500 = sp500_daily[index - length_stock : index]
      beta = stats.linregress(sp_500, stock_returns)[0]
      long_betas.append(beta)


    except:
      long_rets.append(0.0)  
      long_volatilities.append(target / organized_data[portfolios.index(i)]["Stocks"][stock]["Volatility"])
      stock_returns = daily[daily["PERMNO"] == stock]["RET"][days[date - 13] : days[date - 1]]
      length_stock = len(stock_returns)
      index = daily_days.index(days[date - 1]) + 1
      sp_500 = sp500_daily[index - length_stock : index]
      beta = stats.linregress(sp_500, stock_returns)[0]
      long_betas.append(beta)

    count += 1
  long_weights = i["Portfolios"]["Long Portfolio"]["Weights"].tolist()
  long_RET = 0
  long_WEIGHT = 0
  
  beta_long = 0
  for n in range(len(long_weights)):
    long_RET += long_weights[n] * (long_rets[n]) * long_volatilities[n]
    beta_long += long_weights[n] * long_betas[n] * long_volatilities[n]
    long_WEIGHT += long_weights[n] * long_volatilities[n]
  beta_long_historic.append(beta_long)

#Same for short portfolio
  short_stocks = i["Portfolios"]["Short Portfolio"]["Stocks"]
  short_rets = []
  short_betas = []
  short_volatilities = []
  for stock in short_stocks:
    try:
      short_rets.append((1 + monthly[monthly["PERMNO"] == stock][dates[date]]["RET"].values[0]) * (1 + monthly[monthly["PERMNO"] == stock][dates[date]]["DLRET"].values[0]) - 1)
      short_volatilities.append(target / organized_data[portfolios.index(i)]["Stocks"][stock]["Volatility"])
      stock_returns = daily[daily["PERMNO"] == stock]["RET"][days[date - 13] : days[date - 1]]
      length_stock = len(stock_returns)
      index = daily_days.index(days[date - 1]) + 1
      sp_500 = sp500_daily[index - length_stock : index]
      beta = stats.linregress(sp_500, stock_returns)[0]
      short_betas.append(beta)

    except:
      short_rets.append(0.0)  
      short_volatilities.append(target / organized_data[portfolios.index(i)]["Stocks"][stock]["Volatility"])
      stock_returns = daily[daily["PERMNO"] == stock]["RET"][days[date - 13] : days[date - 1]]
      length_stock = len(stock_returns)
      index = daily_days.index(days[date - 1]) + 1
      sp_500 = sp500_daily[index - length_stock : index]
      beta = stats.linregress(sp_500, stock_returns)[0]
      short_betas.append(beta)

  short_weights = i["Portfolios"]["Short Portfolio"]["Weights"].tolist()
  short_RET = 0
  short_WEIGHT = 0 
   
  beta_short = 0
  for n in range(len(short_weights)):
    short_RET += -short_weights[n] * (short_rets[n]) * short_volatilities[n]
    beta_short += short_weights[n] * short_volatilities[n] * short_betas[n]
    short_WEIGHT += -short_weights[n] * short_volatilities[n]
  beta_short_historic.append(beta_short)

  beta_long = abs(beta_long)
  beta_short = abs(beta_short)

  if beta_long < 0.5: #Limit leverage to 200%
    beta_long = 0.5
    long_WEIGHT = long_WEIGHT * 2
  else:
    long_WEIGHT = long_WEIGHT * 1/beta_long
  long_weights_beta_historic.append(long_WEIGHT)

  if beta_short < 0.5:
    beta_short = 0.5
    short_WEIGHT = short_WEIGHT * 2
  else:
    short_WEIGHT = short_WEIGHT * 1/beta_short
  short_weights_beta_historic.append(short_WEIGHT)
  
  
  total_weight = short_WEIGHT + long_WEIGHT

  monthly_return = (1 / beta_long) * long_RET + (1 / beta_short) * short_RET

  
  if abs(total_weight) > 2.0:
    monthly_return = monthly_return * (2 / abs(total_weight)) #if the leverage is higher than 200% or lower than -200%, deleverage the monthly return
  else:
    pass

  if signals[portfolios.index(i)] == "High":
    total_weights_beta_historic.append(0.0)
    port_rets_beta.append(0.0)
  else:
    total_weights_beta_historic.append(long_WEIGHT + short_WEIGHT)   
    port_rets_beta.append(monthly_return)
  print((portfolios.index(i) + 1) / len(portfolios) * 100)

In [ ]:
excess_returns = pd.DataFrame(port_rets_beta[:-1], index = plot_dates[1:], columns = ["Strategy Returns"]) 
excess_returns["Risk Free"] = rf.values
excess_returns["Excess Returns"] = excess_returns["Strategy Returns"] - excess_returns["Risk Free"]
rets = excess_returns["Excess Returns"].values

std = statistics.pstdev(rets) * np.sqrt(12) * 100
mean = np.mean(rets) * 12 * 100
skewness = skew(rets)
kurt = kurtosis(rets)
max = np.max(rets) * 100
min = np.min(rets) * 100
std_negative = statistics.pstdev([ret for ret in rets if ret < 0])  * np.sqrt(12) * 100

print("Standard Deviation: " + str(std) + "%")
print("Average Annual Return: " + str(mean) + "%")
print("Sharpe Ratio: " + str(mean/std))
print("Sortino Ratio: " + str(mean/std_negative))
print("Largest Gain: " + str(max) + "%")
print("Largest Loss: " + str(min) + "%") 
print("Skewness: " + str(skewness))
print("Kurtosis: " + str(kurt))

**To replicate the Performance Statistics and Plots open the respective notebooks that are available in the git repository**